In [ ]:
import pandas as pd
import numpy as np
import tarfile as tar
import re

In [ ]:
groups = ["alt.atheism", "sci.space", "talk.religion.misc", "comp.graphics"]
data = []
labels = []
with tar.open(name="D:/HTWG/MSI/Semester1/Machine Learning/Aufgaben/20news-18828.tar.gz", mode='r:gz') as archive:
    for info in archive:
        if info.isfile() and info.name.split('/')[-2] in groups:
            file = archive.extractfile(info.name)
            data.append(file.read().decode("latin-1"))
            labels.append(info.name.split('/')[-2])

In [ ]:
print(len(data))

In [ ]:
data_partition = [d.partition('\n\n')[-1] for d in data]
print(data_partition[1])
print(data[1])

In [ ]:
tokenized_docs = [re.compile(r"(?u)\b\w\w+\b").findall(partition.lower()) for partition in data_partition]
all_tokens = []
for tokens in tokenized_docs:
    all_tokens.extend(tokens)
unique_tokens = set(all_tokens)
n_tokens = len(unique_tokens)
print(n_tokens)
print(len(unique_tokens))
print(tokenized_docs[1])

In [ ]:
from collections import Counter
vectors = []
for tokens in tokenized_docs:
    counter = Counter(tokens)
    vector = []
    for token in unique_tokens:
        #print(counter[token])
        vector.append(counter[token])
    vectors.append(vector)
print(vectors[0])
    

In [ ]:
cut = int(len(vectors) * 0.6)
test_data = vectors[cut:]
test_labels = labels[cut:]
train_data = vectors[:cut]
train_labels = labels[:cut]
n_train = len(train_data)
n_test = len(test_data)
print(n_train)
print(n_test)

In [ ]:
Ni = {groups[0] : 0, 
       groups[1] : 0, 
       groups[2] : 0, 
       groups[3] : 0}

nij = {groups[0] : np.zeros(n_tokens, dtype=np.int), 
       groups[1] : np.zeros(n_tokens, dtype=np.int), 
       groups[2] : np.zeros(n_tokens, dtype=np.int), 
       groups[3] : np.zeros(n_tokens, dtype=np.int)}

pij = {groups[0] : np.zeros(n_tokens), 
       groups[1] : np.zeros(n_tokens), 
       groups[2] : np.zeros(n_tokens), 
       groups[3] : np.zeros(n_tokens)}

pi = {groups[0] : 0.0, 
      groups[1] : 0.0, 
      groups[2] : 0.0, 
      groups[3] : 0.0}

for i in range(n_train):
    Ni[train_labels[i]] += np.sum(train_data[i])

for i in range(n_train):
    nij[train_labels[i]] += train_data[i]

for group in groups:
    pij[group] = (nij[group] + 1) / (Ni[group] + n_tokens) 
    
for group in groups:
    print("Group: {}".format(group))
    print("Ni: {}".format(Ni[group]))
    print("nij: {}".format(nij[group]))
    print("pij: {}".format(pij[group]))
    print()

In [ ]:
corrects = np.full(shape=(n_test), fill_value=False)
for i, lable in enumerate(test_labels):
    max_p, max_group = -np.inf, None
    for group in groups:
        p = np.sum(np.log(pij[group]) * test_data[i])
        
        if p > max_p:
            max_p = p
            max_group = group
            
    corrects[i] = (lable==max_group)
print("Correct: {}; Total: {}; Accuracy: {}%".format(corrects.sum(), n_test, np.round(corrects.sum() / n_test, decimals=4)*100))

In [ ]:
np.log([1,2])